In [115]:
import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf

In [116]:
# data_set_url = '/home/ireshr/PageroLabs/Full_dataset/secure_archive/dataset.csv'
data_set_url = '/home/ireshr/PageroLabs/11-1/dataset_acrhive/processed-dataset.csv'
b_data=pd.read_csv( data_set_url , lineterminator='\n' , header = None)
b_data=b_data.astype(str)

In [117]:
subject_array = b_data[1][:10000]
description_array = b_data[2][:10000]
teams = b_data[3][:10000]

In [118]:
description_array.size

9999

In [119]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    subject_array + description_array, target_vocab_size=2**18)

In [120]:
VOCAB_SIZE = tokenizer.vocab_size

In [121]:
from datetime import  datetime

now = datetime.now()
tokenizer.save_to_file('./dictionary_'+ now.strftime("%m %d %Y, %H:%M:%S"))

In [122]:
START_TOKEN, END_TOKEN , SUBJECT_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1] , [tokenizer.vocab_size + 2]

In [123]:
def concat_subject_and_description(subjects, descriptions):
    inputs = []
    for (subject, description) in zip(subjects, descriptions):
        input = SUBJECT_TOKEN + subject + SUBJECT_TOKEN + description
        inputs.append(input)
    return inputs

ticket_input = concat_subject_and_description(subject_array, description_array)

TypeError: can only concatenate list (not "str") to list

In [ ]:
def maximum_input_size(inputs):
    max = len(inputs[0])
    for element in inputs:
        max = max if max > len(element) else len(element)
    return max
maximum_input_size(ticket_input)

In [ ]:
MAX_LENGTH = 100 
#  This is just a random value

In [ ]:
# Tokenize, filter and pad sentences
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # tokenize sentence
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        # check tokenized sentence max length
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)

    # pad tokenized sentences
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(ticket_input, teams)